# Create a mpi4py environement and run it <br>across multiple nodes
This notebook shows how you can run MPI programs across multiple nodes<br>
from within a notebook.  We provide a script to build  the environment and<br>
a script to launch it.  We also provide a convience script for creating a <br>
tunnel from your desktop machine to a compute node.

We also show that you can compile mpi programs while you are in the notebook.

## Note this environemnt contains the slurm magic commands, shown below.

## Install slurm magic commands
`
To install the slurm magic commands run the command:
`
```
pip install git+https://github.com/NERSC/slurm-magic.git
```

Here is the source:

https://github.com/NERSC/slurm-magic/blob/master/slurm_magic.py





In [1]:
#we are using my wall clock timer "included"
from tymer import tymer
tymer(["-i","start"])

1620849300.610604 Wed May 12 13:55:00 2021      0.000      0.000 start


## Load slurm magic commands

In [2]:
%load_ext slurm_magic

In [3]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %sacct  %sacctmgr  %salloc  %sattach  %save  %sbatch  %sbcast  %sc  %scancel  %scontrol  %sdiag  %set_env  %sinfo  %slurm  %smap  %sprio  %squeue  %sreport  %srun  %sshare  %sstat  %store  %strigger  %sview  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls

## Here's the script that created an environment with MPI4py.  It also adds TF and CUPY.


In [4]:
cat makeit

#script for build conda environment with various packages including
#jupyter matplotlib scipy pandas xlwt dask tensorflow horovod cupy
#You'll need to change "BASEDIR" then
#
# source makeit
#
CWD=`pwd`
export MYVERSION=may12
#BASEDIR=/scratch/$USER
BASEDIR=/projects/hpcapps/tkaiser
#cd ~
module load conda 2> /dev/null || echo "module load conda failed"
#conda create -p $BASEDIR/exampleenv --name $MYVERSION python=3.8 jupyter matplotlib scipy pandas xlwt dask -y
conda create -p $BASEDIR/$MYVERSION python=3.8 jupyter matplotlib scipy pandas xlwt dask -y

### Don't do conda init
### Just do source activate
source activate 
source activate $BASEDIR/$MYVERSION

which pip
which python

### Install mpi4py
#module load intel-mpi/2020.1.217  2> /dev/null || echo "module load mpi failed"
module load mpt  2> /dev/null || echo "module load mpi failed"
module load gcc/8.4.0  2> /dev/null || echo "module load gcc failed"
pip --no-cache-dir install mpi4py


### Install slurm magic commands 
pip inst

## Here's the script that launched this notebook

In [5]:
cat launch_2

#!/bin/bash
#SBATCH --job-name="jupyter"
#SBATCH --nodes=2
#SBATCH --account=hpcapps
#SBATCH --time=01:00:00
##SBATCH --gpus-per-node=2 
#SBATCH --partition=debug


export BASEDIR=/projects/hpcapps/tkaiser
export MYVERSION=may12

module load conda
module load mpt
module load cuda/11.2   cudnn/8.1.1/cuda-11.2   gcc/8.4.0

### Don't do conda init
### Just do source activate
source activate 
source activate $BASEDIR/$MYVERSION


#export LD_LIBRARY_PATH=/nopt/nrel/apps/cuda/11.2/targets/x86_64-linux/lib:/nopt/nrel/apps/base/2020-05-12/spack/opt/spack/linux-centos7-x86_64/gcc-4.8.5/gcc-6.5.0-aov4u2ocxtqf4bwif4hc7sga4cvccpxm/lib64:/nopt/nrel/apps/base/2020-05-12/spack/opt/spack/linux-centos7-x86_64/gcc-4.8.5/gcc-6.5.0-aov4u2ocxtqf4bwif4hc7sga4cvccpxm/lib:/nopt/nrel/apps/cudnn/8.1.1-cuda-11.2/lib64:/nopt/nrel/apps/cuda/11.2/lib64:/nopt/mpi/mpt-2.23/lib:/home/tkaiser2/libs/zlib-1.2.11/lib:/home/tkaiser2/libs/bzip2-1.0.8/lib:/home/tkaiser2/bin/jdk-13.0.1/lib:/nopt/slurm/current/lib::

export LD

<hr> 

### After the job starts the file ~/jupyter.log will contain the information required to  connect to the notebook.  What we need is the node name, port number, and the html string.  <br><br>The bash function, tunnel, can provide a shortcut for connecting to the notebook.  After it is installed in your bash environment the syntax is 

`
tunnel node_name port_number
`
### After it connects you copy/past the html string from ~/jupyter.log into a browser

<hr> 

### To get tunnel

`
wget https://raw.githubusercontent.com/NREL/HPC/master/slurm/source/tunnel.sh
`

### We're going to get a few examples to play with:

In [6]:
!wget https://raw.githubusercontent.com/NREL/HPC/master/slurm/source/c_ex02.c

--2021-05-12 13:55:03--  https://raw.githubusercontent.com/NREL/HPC/master/slurm/source/c_ex02.c
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1149 (1.1K) [text/plain]
Saving to: ‘c_ex02.c’

100%[======================================>] 1,149       --.-K/s   in 0s      

2021-05-12 13:55:03 (84.3 MB/s) - ‘c_ex02.c’ saved [1149/1149]



In [7]:
!wget  https://raw.githubusercontent.com/NREL/HPC/master/slurm/source/report.py

--2021-05-12 13:55:03--  https://raw.githubusercontent.com/NREL/HPC/master/slurm/source/report.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1565 (1.5K) [text/plain]
Saving to: ‘report.py’

100%[======================================>] 1,565       --.-K/s   in 0.001s  

2021-05-12 13:55:03 (2.62 MB/s) - ‘report.py’ saved [1565/1565]



In [8]:
ls

c_ex02.c                           launch_gpu          report.py
checkpoint                         libcusolver.so.10@  slurm-6819468.out
checkpoints-1.data-00000-of-00001  makeit              slurm-6821711.out
checkpoints-1.index                minst.out           start
dompi.ipynb                        mninstcu.ipynb      tensorflow2_mnist.py*
done                               mnist1.py*          tymer.py*
dpcu.ipynb                         mnist.err
launch_2                           __pycache__/


## Let's get to it:

In [9]:
squeue -u tkaiser2

,JOBID,PARTITION,NAME,USER,ST,TIME,NODES,NODELIST(REASON)
0,6821711,debug,jupyter,tkaiser2,R,1:48,2,r102u[34-35]


<hr> 

### Get a list of our hosts we have available

In [10]:
srun -n 4 --tasks-per-node=2 hostname

'r102u34\nr102u34\nr102u35\nr102u35\n'

In [11]:
out=_
out=out.split()
#print(out)
for o in out :
    print(o)

r102u34
r102u34
r102u35
r102u35


<hr> 

### Run a mpi4py program and put output in a file

In [12]:
!chmod 755 report.py


In [13]:
srun -n 72 --tasks-per-node=36 -o report.out ./report.py

''

In [14]:
cat report.out

xxxxxx Hello from  2 on r102u34   ,   2   
xxxxxx Hello from 38 on r102u35   ,   2   
xxxxxx Hello from 37 on r102u35   ,   1   
xxxxxx Hello from 39 on r102u35   ,   3   
xxxxxx Hello from 40 on r102u35   ,   4   
xxxxxx Hello from 43 on r102u35   ,   7   
xxxxxx Hello from 52 on r102u35   ,  16   
xxxxxx Hello from 41 on r102u35   ,   5   
xxxxxx Hello from 42 on r102u35   ,   6   
xxxxxx Hello from 45 on r102u35   ,   9   
xxxxxx Hello from 46 on r102u35   ,  10   
xxxxxx Hello from 47 on r102u35   ,  11   
xxxxxx Hello from 48 on r102u35   ,  12   
xxxxxx Hello from 49 on r102u35   ,  13   
xxxxxx Hello from 50 on r102u35   ,  14   
xxxxxx Hello from 51 on r102u35   ,  15   
xxxxxx Hello from 53 on r102u35   ,  17   
xxxxxx Hello from 54 on r102u35   ,  18   
xxxxxx Hello from 55 on r102u35   ,  19   
xxxxxx Hello from 56 on r102u35   ,  20   
xxxxxx Hello from 57 on r102u35   ,  21   
xxxxxx Hello from 58 on r102u35   ,  22   
xxxxxx Hello from 59 on r102u35   ,  23   
xxxxxx Hell

<hr> 

### Now we are going to build and run a C mpi program

In [15]:
ls

c_ex02.c                           launch_gpu          report.out
checkpoint                         libcusolver.so.10@  report.py*
checkpoints-1.data-00000-of-00001  makeit              slurm-6819468.out
checkpoints-1.index                minst.out           slurm-6821711.out
dompi.ipynb                        mninstcu.ipynb      start
done                               mnist1.py*          tensorflow2_mnist.py*
dpcu.ipynb                         mnist.err           tymer.py*
launch_2                           __pycache__/


In [16]:
!mpicc c_ex02.c -o c_ex02

In [17]:
srun -n 2 ./c_ex02

'Hello from c process      :    0  Numprocs is    2 r102u34\nHello from c process      :    1  Numprocs is    2 r102u35\ngetting 1\ni= 200 \n'

In [18]:
out=_
out=out.split("\n")
for o in out:
    print(o)

Hello from c process      :    0  Numprocs is    2 r102u34
Hello from c process      :    1  Numprocs is    2 r102u35
getting 1
i= 200 



In [19]:
tymer(["-i","done"])

1620849307.953585 Wed May 12 13:55:07 2021      7.343      7.343 done


<hr> 

### We note that we can also start slurm jobs from within a notebook.  The<br> syntax is simply

`
sbatch script
`

### We could, for example, start a notebook on europa and then launch the <br>batch script given above to start a parallel notebook.

